CIFAR10 데이터셋을 이미지 분류 

컴퓨터 비전 분야에서 딥러닝을 통해 

가장 처음에 고안된 방안인

이미지 분류를 진행해보자!

CIFAR 10 데이터셋은 

3*32*32 픽셀의 60000개의 컬러이미지(RGB) 데이터셋이다.

클래스의 개수는 10개이며 

비행기, 자동차, 새, 고양이, 사슴, 개, 개구리, 말, 배, 트럭이 있다.

보통 50000개는 훈련, 10000개는 테스트데이터셋으로 구성하는 편이다.

* `torchvision` - base domain library for Pytorch computer vision
* `torchvision.datasets` - get datasets and data loading functions for computuer vision here
* `torchvision.models` - get pretrained computer vision models that you can leverage for your own problems
* `torchvision.transform` - functions for manipulating your vision data (images) to be suitable foruse with an ML model
* `torch.utils.data.Dataset` - Base dataset class for Pytorch.
* `torch.utils.data.DataLoader` - Creates a Python iterable over a dataset

In [ ]:
import torch
import torchvision # 컴퓨터 비전에 필요한 패키징

from torchvision import datasets # Image 데이터셋이 있는곳
from torchvision import transforms # 이미지 전처리(Crop, 어파인 변환 이런게 있는거 같던데)
from torch.utils.data import Dataset # 데이터셋 클래스 상속할 때 쓰는 기반클래스
from torch.utils.data import DataLoader # dataset을 iterable할 수 있게하는 Dataloader

import matplotlib.pyplot as plt
# agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

print(device) # gpu 사용

In [ ]:
# 1. Getting Data (데이터셋 가져오기)

train_dataset=datasets.CIFAR10(root='Data',
                               train=True,
                               transform=transforms.ToTensor(),
                               target_transform=None,
                               download=True)
test_dataset=datasets.CIFAR10(root='Data',
                              train=False,
                              transform=transforms.ToTensor(),
                              target_transform=None,
                              download=True)



In [ ]:
# datasets.CIFAR10를 참조한 train_dataset에 대해 확인하기
print(train_dataset)
print(dir(train_dataset)) # 클래스에 부여된 속성 또는 메소드를 확인할 수 있다. 'class_to_idx', 'classes'
print(len(train_dataset))

print(train_dataset.class_to_idx) # 딕셔너리 형태
print(train_dataset.classes) # 리스트 형태 

print(train_dataset.__getitem__(0)) # 인덱스 0번의 값 출력, 튜플 형태 (텐서정보(3*32*32),클래스넘버) 정보 전달


In [ ]:
# 데이터셋 추출 후 시각화 해보기

plt.figure()
plt.imshow(train_dataset.__getitem__(0)[0].permute(1,2,0)) # torch의 텐서와 matplotlib의 rgb 순서가 달라서 변환해줘야함.
plt.title(train_dataset.classes[train_dataset.__getitem__(0)[1]])
plt.axis(False)
plt.show()

In [ ]:
# 랜덤으로 10개 정도 그려보기
import random
fig = plt.figure(figsize=(60,60))
nrows,ncols=1,10

for i in range(1,nrows*ncols+1):
    random_idx = random.randint(a=0, b=len(train_dataset))
    img, idx = train_dataset[random_idx]
    fig.add_subplot(nrows,ncols,i)
    plt.imshow(img.permute(1,2,0))
    plt.axis(False)
    plt.title(train_dataset.classes[idx])
    plt.show()


데이터셋 정보를 iterable한 객체로 만들어서 model에 넣어줘야한다.

DataLoader를 구현해야함

In [31]:
from torch.utils.data import DataLoader
from os import cpu_count
BATCH_SIZE=32
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=cpu_count()
)
test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=cpu_count(),
                             )

print(train_dataloader)
print(len(train_dataloader)) # 1563 (50000//32) shape(32,3,32,32)
print(dir(train_dataloader)) 

train_dataloader_feature,train_dataloader_label=next(iter(train_dataloader))
test_dataloader_feature,test_dataloader_label=next(iter(test_dataloader))
train_dataloader_feature.shape # shape (32,3,32,32) = (batch_size,channel,height,width)


1563
['_DataLoader__initialized', '_DataLoader__multiprocessing_context', '_IterableDataset_len_called', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_auto_collation', '_dataset_kind', '_get_iterator', '_index_sampler', '_is_protocol', '_iterator', 'batch_sampler', 'batch_size', 'check_worker_number_rationality', 'collate_fn', 'dataset', 'drop_last', 'generator', 'multiprocessing_context', 'num_workers', 'persistent_workers', 'pin_memory', 'pin_memory_device', 'prefetch_factor', 'sampler', 'timeout', 'worker_init_fn']


torch.Size([32, 3, 32, 32])

train_dataloader_feature와 train_dataloader_label을 알아보자

In [35]:
print(train_dataloader_feature)
print(train_dataloader_feature.shape)
print(len(train_dataloader_feature))

tensor([[[[0.5647, 0.6039, 0.3137,  ..., 0.5608, 0.6588, 0.7137],
          [0.5647, 0.4235, 0.2039,  ..., 0.6275, 0.5451, 0.6627],
          [0.7137, 0.6235, 0.2235,  ..., 0.6706, 0.6627, 0.5882],
          ...,
          [0.2824, 0.2706, 0.2941,  ..., 0.4863, 0.4039, 0.4314],
          [0.2941, 0.4039, 0.3412,  ..., 0.2902, 0.3451, 0.2431],
          [0.5686, 0.4235, 0.5020,  ..., 0.5059, 0.6314, 0.7137]],

         [[0.3608, 0.4078, 0.1961,  ..., 0.4392, 0.5569, 0.6118],
          [0.3961, 0.2431, 0.0667,  ..., 0.4824, 0.4235, 0.5412],
          [0.6039, 0.4902, 0.0980,  ..., 0.5333, 0.5529, 0.4745],
          ...,
          [0.1294, 0.1176, 0.1412,  ..., 0.3529, 0.2745, 0.3059],
          [0.1333, 0.2314, 0.1765,  ..., 0.1412, 0.2078, 0.1255],
          [0.4078, 0.2510, 0.3529,  ..., 0.3412, 0.4902, 0.5961]],

         [[0.3608, 0.3961, 0.2078,  ..., 0.4196, 0.5373, 0.6000],
          [0.4000, 0.2549, 0.0863,  ..., 0.4745, 0.4118, 0.5373],
          [0.6039, 0.5098, 0.1216,  ..., 0

CIFAR10 데이터셋을 학습시킬 Model 만들어야한다.

TinyVGG (이미지 추출기)를 사용해보자

source - https://poloclub.github.io/cnn-explainer/